In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from ipywidgets import interact, Checkbox, HBox, VBox, Output, Layout


def get_stock_data(stock_name, stock_code, start_date, end_date, split_date=None, split_ratio=None):
    stock_url = f"https://finance.naver.com/item/sise_day.naver?code={stock_code}"
    print("\n" + stock_url)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    stock_data = []
    pages = (end_date - start_date).days // 10 + 1
    
    for page in range(1, pages + 1):
        page_url = f"{stock_url}&page={page}"
        response = requests.get(page_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table")
    
        for row in table.find_all("tr")[1:]:
            cells = row.find_all("td")
            if len(cells) > 1:
                date = cells[0].text.strip()
                price = cells[1].text.strip().replace(",", "")
                if price.isdigit():
                    stock_data.append((date, int(price)))
                    if datetime.strptime(date, "%Y.%m.%d") <= end_date:
                        break
    
    df = pd.DataFrame(stock_data, columns=["Date", stock_name])
    df["Date"] = pd.to_datetime(df["Date"], format="%Y.%m.%d")
    df = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]
    df = df.sort_values("Date").reset_index(drop=True)
    
    if len(df) > 0:
        if split_date and split_ratio:
            split_date = datetime.strptime(split_date, "%Y-%m-%d")
            df.loc[df["Date"] < split_date, stock_name] /= split_ratio
        
        start_price = df.iloc[0][stock_name]
        df[f"{stock_name}_Change"] = (df[stock_name] - start_price) / start_price * 100
    
    return df

def get_kospi_data(start_date, end_date):
    kospi_data = []
    pages = (end_date - start_date).days // 6 + 1
    stock_url = f"https://finance.naver.com/sise/sise_index_day.naver?code=KOSPI"
    print("\n" + stock_url)

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    for page in range(1, pages + 1):
        page_url = f"{stock_url}&page={page}"
        response = requests.get(page_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table")
        
        for row in table.find_all("tr")[1:]:
            cells = row.find_all("td")
            if len(cells) > 1:
                date = cells[0].text.strip()
                price = cells[1].text.strip().replace(",", "")
                if price.replace('.', '', 1).isdigit():  # 소수점을 포함한 숫자 체크
                    kospi_data.append((date, float(price)))  # float으로 변환
                    if datetime.strptime(date, "%Y.%m.%d") <= end_date:
                        break

    df = pd.DataFrame(kospi_data, columns=["Date", "KOSPI"])
    df["Date"] = pd.to_datetime(df["Date"], format="%Y.%m.%d")
    df = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]
    df = df.sort_values("Date").reset_index(drop=True)
    
    if len(df) > 0:
        start_price = df.iloc[0]["KOSPI"]
        df["KOSPI_Change"] = (df["KOSPI"] - start_price) / start_price * 100
    
    return df

def plot_stocks(selected_stocks):
    plt.figure(figsize=(12, 6))
    for stock_name in selected_stocks:
        df = stock_dfs[stock_name]
        if not df.empty:
            plt.plot(df["Date"], df[f"{stock_name}_Change"], label=stock_name, linestyle='-')

    plt.title("Change from Start Date")
    plt.xlabel("Date")
    plt.ylabel("Change (%)")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 6, 1)


stock_codes = {   
    "KOSPI": {"code": "KOSPI"},
}


stock_dfs = {}
for stock_name, info in stock_codes.items():
    stock_code = info["code"]
    split_date = info.get("split_date")
    split_ratio = info.get("split_ratio")
    if stock_code == "KOSPI":
        stock_dfs[stock_name] = get_kospi_data(start_date, end_date)
    else:
        stock_dfs[stock_name] = get_stock_data(stock_name, stock_code, start_date, end_date, split_date, split_ratio)

stock_options = {stock_name: Checkbox(value=True, description=stock_name) for stock_name in stock_dfs.keys()}
ui = VBox([HBox(list(stock_options.values()))])
output = Output()

def update_plot(*args):
    with output:
        output.clear_output(wait=True)
        selected_stocks = [stock for stock, checkbox in stock_options.items() if checkbox.value]
        plot_stocks(selected_stocks)

for checkbox in stock_options.values():
    checkbox.observe(update_plot, 'value')

display(ui, output)
update_plot()




https://finance.naver.com/item/sise_day.naver?code=012200

https://finance.naver.com/sise/sise_index_day.naver?code=KOSPI


Output()